In [1]:
import os
import sys
sys.path.insert(0, './utils/')
from utils import load_data, normalize_data, format_data, split_data
from utils import train_model, plot_train_val, forecast, forecast_analysis, experiment

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, TimeDistributed, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, LSTM, Bidirectional, Conv1D, Permute
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from copy import copy
from tensorflow.keras import backend as K
from sklearn import preprocessing
import tensorflow.keras.optimizers as optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

In [2]:
def experiment(model_fn, hyperp):
    K.clear_session()
    
    # Load and format_data
    df = load_data(hyperp['lookback_window'], city= hyperp['city'])
    x, y = format_data(df, hyperp['lookback_window'], lstm=True)
    
    if 'normalize' in hyperp and hyperp['normalize']:
        df = normalize_data(df)
    
    val_size = int(math.floor((len(x)*(1-hyperp['train_split_percent']))))
    x_train, y_train, x_val, y_val = split_data(x, y, val_size)
    data = x_train, y_train, x_val, y_val
    
    # Train Model
    input_shape = (x.shape[1], x.shape[2])
    model = model_fn(input_shape)
    train_model(model, data, epochs=hyperp['epochs'], verbose=hyperp['verbose'])
    plot_train_val(model, data)
    df_results = forecast_analysis(model, data)
    return df_results

## Separate splits

## Long Short-Term Memory (LSTM) model

In [5]:
def get_lstm_model(input_shape):
    i = Input(shape=(input_shape))
    print(i.shape)
    m = i
#     m = Permute((2, 1))(m)
    m = Dropout(0.05)(m)
    m = LSTM(50, activation='relu')(m)
    m = Dense(500, activation='relu')(m)
    m = Dense(200, activation='relu')(m)
    m = Dense(1, activation='relu')(m)
    
    model = Model(inputs=[i], outputs=[m])

    optimizer=optimizers.Adam(lr=5e-6)
    model.compile(optimizer, 'mae')
    return model

## San Juan

In [ ]:
hyperparameters = {
    'train_split_percent': 0.65,
    'lookback_window': 170,
    'normalize': True,
    'epochs': 250,
    'verbose': 2,
    'city':'sj'
}
df_results_sj = experiment(get_lstm_model, hyperparameters)

(?, 170, 23)
Training...
Train on 498 samples, validate on 268 samples
Epoch 1/250
498/498 - 3s - loss: 2451184.4468 - val_loss: 1114216.5042
Epoch 2/250
498/498 - 2s - loss: 1963957.8258 - val_loss: 908647.1059
Epoch 3/250
498/498 - 2s - loss: 1498899.8760 - val_loss: 707120.8186
Epoch 4/250
498/498 - 2s - loss: 1076964.3233 - val_loss: 452865.8092
Epoch 5/250
498/498 - 2s - loss: 620945.0850 - val_loss: 271002.5386
Epoch 6/250
498/498 - 2s - loss: 242062.2374 - val_loss: 62102.4842
Epoch 7/250
498/498 - 2s - loss: 39.3576 - val_loss: 90909.5953
Epoch 8/250
498/498 - 2s - loss: 30552.0118 - val_loss: 40655.1284
Epoch 9/250
498/498 - 2s - loss: 1214.0999 - val_loss: 49652.0227
Epoch 10/250
498/498 - 2s - loss: 39.1843 - val_loss: 50313.7793
Epoch 11/250
498/498 - 2s - loss: 51533.3859 - val_loss: 45867.8580
Epoch 12/250
498/498 - 2s - loss: 42.0322 - val_loss: 2512.8142
Epoch 13/250
498/498 - 2s - loss: 39.1707 - val_loss: 55.4625
Epoch 14/250
498/498 - 2s - loss: 28909.3046 - val_loss

## Iquitos

In [ ]:
hyperparameters = {
    'train_split_percent': 0.65,
    'lookback_window': 150,
    'normalize': True,
    'epochs': 400,
    'verbose': 2,
    'city':'iq'
}
df_results_iq = experiment(get_lstm_model, hyperparameters)

In [ ]:
## inverse normalize error 

new_df = load_data(lookback_window_sj, city='sj')
nx = new_df.total_cases.values.reshape(-1, 1) #returns a numpy array
min_max_scaler_2 = preprocessing.MinMaxScaler()
min_max_scaler_2.fit(nx)

error = min_max_scaler_2.inverse_transform(np.array(0.023616499232121283).reshape(-1, 1))
print("SJ Forecast MAE: {}".format(error))

ndf = load_data(lookback_window_iq, city='iq')
nx2 = ndf.total_cases.values.reshape(-1, 1) #returns a numpy array
min_max_scaler_3 = preprocessing.MinMaxScaler()
min_max_scaler_3.fit(nx2)

error = min_max_scaler_2.inverse_transform(np.array(0.06704732649868755).reshape(-1, 1))
print("IQ Forecast MAE: {}".format(error))